In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.utils import shuffle

import sys 
sys.path.append('../src/utils/')
sys.path.append('../src/models/')
sys.path.append('../src/visualization/')

from particles import Photon
from seed_finder import SeedFinder

%load_ext autoreload
%autoreload 2

# Load and prepare data for training from one- and two-particle samples

In [2]:
photon_1 = Photon(n_pcl=1)
photon_2 = Photon(n_pcl=2)

photons = [photon_1, photon_2]

In [3]:
Xtrain, ytrain = [], []
Xvalid, yvalid = [], []

for photon in photons:
    Xi, yi = photon.data_for_seed_finder("train")
    Xtrain.extend(Xi)
    ytrain.extend(yi)

    Xi, yi = photon.data_for_seed_finder("valid")
    Xvalid.append(Xi)
    yvalid.append(yi)
Xtrain, ytrain = np.array(Xtrain), np.array(ytrain)
Xvalid, yvalid = np.array(Xvalid), np.array(yvalid)

100%|██████████| 100000/100000 [00:19<00:00, 5255.16it/s]


(938472,) (938472, 7, 7, 1)


100%|██████████| 50000/50000 [00:12<00:00, 4007.99it/s]


(740676,) (740676, 7, 7, 1)


In [4]:
# shuffle the samples randomly
Xvalid, ytrain = shuffle(Xtrain, ytrain, random_state=42)
Xvalid, yvalid = shuffle(Xvalid, yvalid, random_state=42)

# Prepare the model and training

In [7]:
model = SeedFinder().architecture()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy())
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 7, 7, 1)]         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 128)         1280      
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 3, 256)         295168    
                                                                 
 flatten_1 (Flatten)         (None, 2304)              0         
                                                                 
 dense_3 (Dense)             (None, 2400)              5532000   
                                                                 
 dense_4 (Dense)             (None, 500)               1200500   
                                                                 
 dense_5 (Dense)             (None, 1)                 501 

In [8]:
filepath = 'models/seed_finder/run_1/'

checkpoint = ModelCheckpoint(
        str(filepath)+"/weights.{epoch:02d}-{val_loss:.4f}.hdf5", 
        monitor='val_loss', verbose = 1, save_best_only = True, 
        mode='auto', save_freq='epoch')

callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)

In [ ]:
history = model.fit(Xtrain, ytrain,
                    validation_data=(Xvalid, yvalid), 
                    epochs=500, batch_size=64, 
                    callbacks=[checkpoint, callback])

In [ ]:
model.save(str(filepath))

# Draw loss function 

import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = (5,5)
plt.rcParams['figure.dpi'] = 150

plt.scatter(np.arange(0,500), history.history["loss"], s=1., label='loss')
plt.scatter(np.arange(0,500), history.history["val_loss"], s=1., label='val_loss')
plt.legend()
plt.savefig(str(filepath) + '/loss_function')
#plt.xlim(0,500)
#plt.ylim(0.,0.01)